# **Variational Autoencoder with a Normalizing Flow prior** ${\P}$ 

In [1]:
import pyro
import torch
import torch.nn as nn
import torch.utils.data as data
import zuko

from pyro.contrib.zuko import ZukoToPyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
from torch import Tensor
from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_tensor, to_pil_image
from tqdm import tqdm

## Data

In [2]:
trainset = MNIST(root='./', download=True, train=True, transform=to_tensor)
trainloader = data.DataLoader(trainset, batch_size=256, shuffle=True)

In [3]:
x = [trainset[i][0] for i in range(16)]
x = torch.cat(x, dim=-1)

to_pil_image(x)

## Model

In [4]:
class GaussianEncoder(nn.Module):
    def __init__(self, features: int, latent: int):
        super().__init__()

        self.hyper = nn.Sequential(
            nn.Linear(features, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2 * latent),
        )

    def forward(self, x: Tensor):
        phi = self.hyper(x)
        mu, log_sigma = phi.chunk(2, dim=-1)

        return pyro.distributions.Normal(mu, log_sigma.exp()).to_event(1)


class BernoulliDecoder(nn.Module):
    def __init__(self, features: int, latent: int):
        super().__init__()

        self.hyper = nn.Sequential(
            nn.Linear(latent, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, features),
        )

    def forward(self, z: Tensor):
        phi = self.hyper(z)
        rho = torch.sigmoid(phi)

        return pyro.distributions.Bernoulli(rho).to_event(1)

In [5]:
class VAE(nn.Module):
    def __init__(self, features: int, latent: int = 16):
        super().__init__()

        self.encoder = GaussianEncoder(features, latent)
        self.decoder = BernoulliDecoder(features, latent)

        self.prior = zuko.flows.MAF(
            features=latent,
            transforms=3,
            hidden_features=(256, 256),
        )

    def model(self, x: Tensor):
        """
        pyro.sample 定义从概率分布中生成数据或潜变量。
        """
        pyro.module("prior", self.prior)
        pyro.module("decoder", self.decoder)

        with pyro.plate("batch", len(x)):
            z = pyro.sample("z", ZukoToPyro(self.prior()))
            x = pyro.sample("x", self.decoder(z), obs=x)

    def guide(self, x: Tensor):
        """
        pyro.sample 定义近似分布，用来推断 model 中的潜变量。
        """
        pyro.module("encoder", self.encoder)

        with pyro.plate("batch", len(x)):
            z = pyro.sample("z", self.encoder(x))

In [7]:
vae = VAE(784, 16).cpu()
vae

VAE(
  (encoder): GaussianEncoder(
    (hyper): Sequential(
      (0): Linear(in_features=784, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=1024, bias=True)
      (3): ReLU()
      (4): Linear(in_features=1024, out_features=32, bias=True)
    )
  )
  (decoder): BernoulliDecoder(
    (hyper): Sequential(
      (0): Linear(in_features=16, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=1024, bias=True)
      (3): ReLU()
      (4): Linear(in_features=1024, out_features=784, bias=True)
    )
  )
  (prior): MAF(
    (transform): LazyComposedTransform(
      (0): MaskedAutoregressiveTransform(
        (base): MonotonicAffineTransform()
        (order): [0, 1, 2, 3, 4, ..., 11, 12, 13, 14, 15]
        (hyper): MaskedMLP(
          (0): MaskedLinear(in_features=16, out_features=256, bias=True)
          (1): ReLU()
          (2): MaskedLinear(in_features=256, out_features=256, bias=True)
      

In [9]:
pyro.clear_param_store()

svi = SVI(vae.model, vae.guide, Adam({'lr': 1e-3}), loss=Trace_ELBO())

for epoch in (bar := tqdm(range(96))):
    losses = []

    for x, _ in trainloader:
        x = x.round().flatten(-3).cpu()

        losses.append(svi.step(x))

    losses = torch.tensor(losses)

    bar.set_postfix(loss=losses.sum().item() / len(trainset))

  1%|          | 1/96 [00:42<1:07:55, 42.90s/it, loss=180]


KeyboardInterrupt: 